<a href="https://colab.research.google.com/github/KU81-ComSci-ProjectEnd-GameCard/main/blob/master/Predictor/data/json_unzip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load data
**https://drive.google.com/drive/folders/1c7MwTdLxnPgvmPbBEfNWa45YAUU53H0l** make sure to load data form this before processing.  

raw_data file so large to push to github

In [8]:
import re
def clear_SearingBlowPlus(lst):
    return [re.sub(r'Searing Blow\+\d+', 'Searing Blow', item) for item in lst]

In [9]:
def upgrade_lst(lst):
  new_lst = list()
  for item in lst:
    if(item == 'Searing Blow'):
      new_lst.append('Searing Blow')
    else:
      new_lst.append(item+'+1')
  return new_lst

In [10]:
def get_ALL_CARD():
  IC_ATTACK_CARDS = ["Strike_R", "Bash", "Anger", "Body Slam", "Clash", "Cleave", "Clothesline", "Headbutt", "Heavy Blade", "Iron Wave", "Perfected Strike", "Pommel Strike", "Sword Boomerang", "Thunderclap", "Twin Strike", "Wild Strike", "Blood for Blood", "Carnage", "Dropkick", "Hemokinesis", "Pummel", "Rampage", "Reckless Charge", "Searing Blow", "Sever Soul", "Uppercut", "Whirlwind", "Bludgeon", "Feed", "Fiend Fire", "Immolate", "Reaper"]
  IC_SKILL_CARDS = ["Defend_R", "Armaments", "Flex", "Havoc", "Shrug It Off", "True Grit", "Warcry", "Battle Trance", "Bloodletting", "Burning Pact", "Disarm", "Dual Wield", "Entrench", "Flame Barrier", "Ghostly Armor", "Infernal Blade", "Intimidate", "Power Through", "Rage", "Second Wind", "Seeing Red", "Sentinel", "Shockwave", "Spot Weakness", "Double Tap", "Exhume", "Impervious", "Limit Break", "Offering"]
  IC_POWER_CARDS = ["Combust", "Dark Embrace", "Evolve", "Feel No Pain", "Fire Breathing", "Inflame", "Metallicize", "Rupture", "Barricade", "Berserk", "Brutality", "Corruption", "Demon Form", "Juggernaut"]
  IRONCLAD_CARDS = IC_ATTACK_CARDS + IC_SKILL_CARDS + IC_POWER_CARDS

  COLORLESS_CARDS = ["Dramatic Entrance", "Flash of Steel", "Mind Blast", "Swift Strike", "HandOfGreed", "Bite", "Expunger", "Ritual Dagger", "Shiv", "Smite", "Through Violence", "Bandage Up", "Blind", "Dark Shackles", "Deep Breath", "Discovery", "Enlightenment", "Finesse", "Forethought", "Good Instincts", "Impatience", "Jack Of All Trades", "Madness", "Panacea", "PanicButton", "Purity", "Trip", "Apotheosis", "Chrysalis", "Master of Strategy", "Metamorphosis", "Secret Technique", "Secret Weapon", "The Bomb", "Thinking Ahead", "Transmutation", "Violence", "Apparition", "Beta", "Insight", "J.A.X.", "Miracle", "Safety", "Magnetism", "Mayhem", "Panache", "Sadistic Nature"]

  CURSE_CARDS = ["AscendersBane", "Clumsy", "Curse of the Bell", "Decay", "Doubt", "Injury", "Necronomicurse", "Normality", "Pain", "Parasite", "Pride", "Regret", "Shame", "Writhe"]
  return IRONCLAD_CARDS + COLORLESS_CARDS + CURSE_CARDS + upgrade_lst(IRONCLAD_CARDS) + upgrade_lst(COLORLESS_CARDS)

In [11]:
import pandas as pd
def get_ic_run(json_data):
  df = pd.DataFrame(json_data)
  runInfo = df['event'].apply(pd.Series)
  runInfo['master_deck'] = runInfo['master_deck'].apply(clear_SearingBlowPlus)
  ## รับแค่ run ของ ironclad
  ALL_CARD = get_ALL_CARD()
  ic_run = runInfo[runInfo['master_deck'].apply(lambda x: all(word in ALL_CARD for word in x))]
  ## เกณคือ ต้องผ่านชั้น 50 แล้ว
  return ic_run.loc[ic_run.floor_reached <= 20]

In [12]:
import os
import gzip
import json

def get_quality_run(directory, run_num,start=0, end=None):
    quality_run = pd.DataFrame()
    try:
        counter = 0
        for file_name in os.listdir(directory):
            file_path = os.path.join(directory, file_name)
            if os.path.isfile(file_path) and file_name.endswith('.json.gz'):
                if counter >= start and (end is None or counter < end):
                    with gzip.open(file_path, 'rt', encoding='utf-8') as gz_file:
                        run_info = json.load(gz_file)
                        ic_run = get_ic_run(run_info)
                        quality_run = pd.concat([quality_run, ic_run], axis=0, join='outer')
                        quality_run.drop_duplicates(subset=['play_id'])
                        print(counter, '   :     ',len(quality_run))
                        if(len(quality_run) >= run_num):
                            break
            counter += 1
    except Exception as e:
        print(f"Error: {e}")
    return quality_run
                        

In [13]:
raw_data = 'Monthly_2020_11'

In [14]:
quality_run = get_quality_run(raw_data, 2000)

0    :      125
1    :      269
2    :      459
3    :      629
4    :      799
5    :      970
6    :      1162
7    :      1333
8    :      1534
9    :      1726
10    :      1938
11    :      2139


In [15]:
quality_run = quality_run.reset_index()

In [16]:
import os
file_path = os.path.join('processed_data', 'bad_run2k.json')
quality_run.to_json(file_path)

In [17]:
(quality_run.loc[quality_run.ascension_level >= 10]).iloc[0].master_deck

['AscendersBane',
 'Strike_R',
 'Strike_R+1',
 'Strike_R',
 'Strike_R+1',
 'Strike_R',
 'Defend_R',
 'Defend_R',
 'Defend_R',
 'Bash+1',
 'Metallicize+1']